In [1]:
import sys
import os
import itertools
import numpy as np
import torch as tr
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
import pprint

import multiprocessing as mp
import concurrent

from utils import *

%load_ext autoreload
%reload_ext autoreload
%autoreload 2


# fit pwm5
- learning rate and critic loss weight

## notes
- PWM0: is the trivial task implementation
    - two stim, response given by second
- PWM5: is closer to real PWM
    - 5 stimuli (ABCDE) 
    - A&E are edge cases where response given by second stim
    - others require comparison

In [2]:
def run_exp(args,neps):
  """ loss [(value, policy),neps] """
  # task and agent definition
  agent_kw = {**args}
  agent = ActorCritic(**agent_kw)
  task_kw = {'stimset':'pwm5','epoch_len':60,'trlen':3}
  task = PWMTaskFR(**task_kw)
  # init loop vars
  reward = -np.ones(neps)
  loss = -np.ones([2,neps])
  pism = -np.ones([3,neps])
  L = []
  # loop over epochs
  for epoch in range(neps):
    epoch_data = run_epoch_FR(agent,task)
    epoch_data = process_epdata(epoch_data)
    update_data = agent.update(epoch_data)
    reward[epoch] = np.sum(epoch_data['reward'])/task.ntrials
    loss[:,epoch] = list(update_data.values())
  return loss,reward



In [ ]:
# RUN GRIDSEARCH
lwL = [2,3,4,5]
ne = 30000
learn_rate = 0.005
gamma = 1.0
lossL = np.zeros([len(lwL),2,ne])
rewardL = np.zeros([len(lwL),ne])
for idx in range(len(lwL)):
  print(lwL[idx])
  argD = {'lweight':lwL[idx],'gamma':gamma,'learnrate':learn_rate}
  lossL[idx],rewardL[idx] = run_exp(argD,ne)
  

2
3


In [ ]:
# PLT GRIDSEARCH
f,axar = plt.subplots(len(lossL),3,figsize=(35,24),sharex=True)
ma_win = 10
labL = ['value','policy']
for li in range(len(lossL)):
  loss = lossL[li]
  axa = axar[li]
  axa[0].set_ylabel('vloss weight = %.3f'%lwL[li])
  for i in range(2):
    ax = axa[i]
    ax.set_title(labL[i])
    ax.plot(loss[i].reshape(-1,ma_win).mean(-1))
    ax.axhline(0,c='k',lw=0.5)
  ax = axa[2]
  ax.plot(rewardL[li].reshape(-1,ma_win).mean(-1))
  ax.set_title('reward')
  ax.set_ylim(0.2,1)
plt.savefig('figures/pwm5-eplen60-vweight_loss-REINFORCE-lrate_%.4f-1seed.png'%learn_rate)